In [13]:
import pandas as pd
from tqdm import tqdm
from sklearn import preprocessing
from xgboost import XGBClassifier

In [14]:
train_data = pd.read_csv('../Database/train_preprocessed.csv')

In [15]:
train_data.head(3)

,ID,대출등급,대출금액,연간소득,부채_대비_소득_비율,총계좌수,총상환원금,총상환이자,대출기간,근로기간,주택소유상태,대출목적
0,TRAIN_00000,2,-0.563848,-0.220218,-0.014287,-0.852449,-0.800303,-0.972784,0,6,3,1
1,TRAIN_00001,1,-0.377964,0.370332,0.087890,-0.356109,-0.436814,-0.441082,1,10,1,10
2,TRAIN_00002,0,-0.610319,0.020823,-0.321114,-0.935172,0.103276,-0.627621,0,5,1,1


대출기간 : 36 months , 60 months   </br>
주택소유상태 : MORTGAGE,RENT,own,ANY </br>
대출목적 : 부채 통합 ,신용 카드,주택 개선 ,기타 ,주요 구매  ,의료 ,자동차,소규모 사업 ,이사 ,휴가, 주택 ,재생 에너지          
     
       

 

In [16]:
label_encoder = preprocessing.LabelEncoder()
train_data['대출기간'] = label_encoder.fit_transform(train_data['대출기간'] )
train_data['주택소유상태'] = label_encoder.fit_transform(train_data['주택소유상태'] )
train_data['대출목적'] = label_encoder.fit_transform(train_data['대출목적'] )


In [17]:
train_data.head(3)

,ID,대출등급,대출금액,연간소득,부채_대비_소득_비율,총계좌수,총상환원금,총상환이자,대출기간,근로기간,주택소유상태,대출목적
0,TRAIN_00000,2,-0.563848,-0.220218,-0.014287,-0.852449,-0.800303,-0.972784,0,6,3,1
1,TRAIN_00001,1,-0.377964,0.370332,0.087890,-0.356109,-0.436814,-0.441082,1,10,1,10
2,TRAIN_00002,0,-0.610319,0.020823,-0.321114,-0.935172,0.103276,-0.627621,0,5,1,1


근로기간

In [18]:
train_data['근로기간'].value_counts()

근로기간
10    32481
1     14449
2      8450
3      7670
0      5671
5      5665
4      5588
8      4888
6      3874
7      3814
9      3744
Name: count, dtype: int64

In [19]:
train_data['근로기간'] = train_data['근로기간'].str.extract(r'(\d+)')
train_data['근로기간']  =train_data['근로기간'].fillna(0)
train_data['근로기간']  =train_data['근로기간'].astype(int)

AttributeError: Can only use .str accessor with string values!

In [ ]:
train_data['근로기간']

main processing

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [10]:
train_data = train_data.drop(columns='ID')
train_data_target = train_data['대출등급']
train_data = train_data.drop(columns='대출등급')

In [20]:
train_data_x, valid_data_x, train_data_y, valid_data_y= train_test_split(train_data,train_data_target,test_size=0.2, shuffle=True,random_state=34)

In [8]:
train = pd.read_csv('../Database/train_preprocessed.csv', index_col='ID')
X = train.drop(columns=['대출등급'])
y = train['대출등급']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## **모델적용**

In [14]:
import optuna
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np

# 데이터 준비 (X_train, y_train 등은 사전에 준비되어야 합니다)
# 예: X_train, X_test, y_train, y_test = train_test_split(...)

def objective(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 10000),
        'max_depth': trial.suggest_int('max_depth', 1, 20),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_float('gamma', 0.1, 5),
        'learning_rate': trial.suggest_float('learning_rate', 0.0001, 0.1),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.01, 1),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.01, 1),
        'subsample': trial.suggest_float('subsample', 0.5, 1)
    }

    model = XGBClassifier(**param, random_state=1004)
    model.fit(train_data_x, train_data_y)
    preds = model.predict(valid_data_x)
    accuracy = accuracy_score(valid_data_y, preds)
    return accuracy

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)  # 예를 들어 100번의 시도

print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)


[I 2024-01-28 17:00:37,829] A new study created in memory with name: no-name-be1114ac-4f91-44dc-9818-2b899f37d88f
[W 2024-01-28 17:00:37,865] Trial 0 failed with parameters: {'n_estimators': 796, 'max_depth': 7, 'min_child_weight': 9, 'gamma': 3.458857253437448, 'learning_rate': 0.0012020810552411008, 'colsample_bytree': 0.8077144331455683, 'reg_lambda': 0.29112381318541636, 'reg_alpha': 0.064974594787714, 'subsample': 0.9710099397715528} because of the following error: ValueError("Invalid classes inferred from unique values of `y`.  Expected: [0 1 2 3 4 5 6], got ['A' 'B' 'C' 'D' 'E' 'F' 'G']").
Traceback (most recent call last):
  File "C:\Users\asas4\anaconda3\envs\python_311\Lib\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\asas4\AppData\Local\Temp\ipykernel_6364\2344010718.py", line 24, in objective
    model.fit(train_data_x, train_data_y)
  File "C:\Users\asas4\anaconda3\envs

ValueError: Invalid classes inferred from unique values of `y`.  Expected: [0 1 2 3 4 5 6], got ['A' 'B' 'C' 'D' 'E' 'F' 'G']

In [9]:
model = XGBClassifier(random_state=1004, n_estimators=8228, max_depth=12, min_child_weight=1, gamma=3, learning_rate=0.0001,
                      colsample_bytree=1.0, reg_lambda=0.767800554, reg_alpha=0.01232, subsample=0.6, device='cuda')
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=1.0, device='cuda', early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=3, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.0001, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=12, max_leaves=None,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=8228, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [11]:
print("CatBoost Accuracy:", accuracy_score(y_test, model.predict(X_test)))

CatBoost Accuracy: 0.8426190352562438


## **valid set도 train에 포함**

In [24]:
model = RandomForestClassifier(n_estimators=2000,max_depth=40, criterion='entropy', random_state=42, n_jobs=-1)
model.fit(train_data, train_data_target)

RandomForestClassifier(criterion='entropy', max_depth=40, n_estimators=2000,
                       n_jobs=-1, random_state=42)

## **test**

In [20]:
test_data = pd.read_csv('../Database/test.csv')
test_data.head(2)

,ID,대출금액,대출기간,근로기간,주택소유상태,연간소득,부채_대비_소득_비율,총계좌수,대출목적,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,연체계좌수
0,TEST_00000,16800000,36 months,8 years,MORTGAGE,132000000,19.64,12,주택 개선,0,394692,146604.0,0.0,0.0
1,TEST_00001,8400000,36 months,5 years,RENT,89971200,15.84,25,부채 통합,0,0,0.0,0.0,0.0


In [21]:
test_data['대출기간'] = label_encoder.fit_transform(test_data['대출기간'] )
test_data['주택소유상태'] = label_encoder.fit_transform(test_data['주택소유상태'] )
test_data['대출목적'] = label_encoder.fit_transform(test_data['대출목적'] )
test_data['근로기간'] = test_data['근로기간'].str.extract(r'(\d+)')
test_data['근로기간']  =test_data['근로기간'].fillna(0)
test_data['근로기간']  =test_data['근로기간'].astype(int)

In [22]:
test_data = test_data.drop(columns='ID')

In [23]:
test_data

,대출금액,대출기간,근로기간,주택소유상태,연간소득,부채_대비_소득_비율,총계좌수,대출목적,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,연체계좌수
0,16800000,0,8,0,132000000,19.64,12,11,0,394692,146604.0,0.0,0.0
1,8400000,0,5,2,89971200,15.84,25,2,0,0,0.0,0.0,0.0
2,17280000,0,6,2,150000000,8.41,20,4,0,1786980,281820.0,0.0,0.0
3,14400000,0,5,0,66000000,13.72,30,4,1,669024,281724.0,0.0,0.0
4,27600000,0,5,2,55200000,30.50,12,4,0,1250052,614844.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
64192,30000000,0,3,0,78000000,22.08,27,2,2,1307532,763380.0,0.0,0.0
64193,30000000,1,10,0,109200000,12.06,26,2,0,960612,1245252.0,0.0,0.0
64194,6120000,0,10,2,39600000,28.80,33,2,0,131520,80880.0,0.0,0.0
64195,11520000,0,10,0,66000000,25.44,41,2,1,1339536,601872.0,0.0,0.0


In [28]:
test_pred = model.predict(test_data)

submissionfile 생성

In [29]:
submission_data = pd.read_csv('./sample_submission.csv')
submission_data['대출등급'] = test_pred
submission_data.to_csv('./randomforest.csv', index=False)


In [30]:
submission_data

,ID,대출등급
0,TEST_00000,B
1,TEST_00001,B
2,TEST_00002,A
3,TEST_00003,C
4,TEST_00004,C
...,...,...
64192,TEST_64192,D
64193,TEST_64193,D
64194,TEST_64194,C
64195,TEST_64195,C
